In [2]:
import wfdb
import os
import pandas as pd
import wfdb.processing as wp
import numpy as np
import pickle
from biosppy.signals import ecg, tools


import torch
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
from torch import nn, optim

import pytorch_model_summary

from sklearn.preprocessing import MinMaxScaler as mms

import matplotlib.pyplot as plt
import matplotlib

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIbLE_DEVICES"] = "0"

# Data Load (using WFDB)

In [3]:
pickle_path = "./mit_pickle"

In [4]:
def record_extract(input_path):
    records = open(input_path+"RECORDS","r")
    records_list = []
    for l in records:
        l = l.rstrip()
        records_list.append(l)
    records.close()
    return records_list

In [5]:
#data extract from Physionet

input_path = "../physionet/mit-bih_arr/1.0.0/"
records = open(input_path+"RECORDS","r")
records_list = []
for l in records:
    l = l.rstrip()
    records_list.append(l)
records.close()
print(records_list)

['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124', '200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '217', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']


In [12]:
NORMAL_ANN = ['N', 'L', 'R', 'e', 'j']
SUPRA_ANN = ['A', 'a', 'J', 'S']
VENTRI_ANN = ['V', 'E']
FUSION_ANN = ['F']
UNCLASS_ANN = ['/', 'f', 'Q']

In [13]:
hb = ecg.extract_heartbeats(signal=record_sig[0][:,0], rpeaks=record_ann, sampling_rate=360,before=0.3,after=0.3)

NameError: name 'record_sig' is not defined

rpeaks_ch = ecg.christov_segmenter(signal=record_sig[0][:,0], sampling_rate=360)

rpeaks_ha = ecg.hamilton_segmenter(signal=record_sig[0][:,0], sampling_rate=360)

comp = ecg.compare_segmentation(rpeaks_ha["rpeaks"],record_ann,sampling_rate=360)

count = 0
for i in comp["match"]:
    if i == count:
        pass
    else:
        print("Missing match I,",i,count)
    count+=1

len(beat["Beats"]),len(beat["rpeaks"])

In [10]:
beat["symbol"]=record_ann_sym[:-1]

NameError: name 'record_ann_sym' is not defined

In [8]:
from sklearn.preprocessing import MinMaxScaler as mms
beat[""]

fig = plt.figure(figsize=(16, 7))
ax = fig.add_subplot(1,1,1)
ax.set_title("Heart Beat")
ax.plot(beat["Beats"][1080])

NameError: name 'beat' is not defined

In [52]:
window_size = 320
def extract_from_sbj(idx):
    record_sig = wfdb.rdsamp(input_path+records_list[idx], channels=[0], sampfrom=0)
    record_ann = list(wfdb.rdann(input_path+records_list[idx],"atr", sampfrom=0).sample)[1:]
    record_ann_sym = list(wfdb.rdann(input_path+records_list[idx], "atr", sampfrom=0).symbol)[1:]
    interval = wp.ann2rr(input_path+records_list[idx], "atr", as_array=True)
    
    rpeaks_ch = ecg.christov_segmenter(signal=record_sig[0][:,0], sampling_rate=360)
    rpeaks_ha = ecg.hamilton_segmenter(signal=record_sig[0][:,0], sampling_rate=360)
    
    #h_beat = ecg.extract_heartbeats(signal=record_sig[0][:,0], rpeaks=rpeaks_ha, sampling_rate=360)
    NORMAL_ANN = ['N', 'L', 'R', 'e', 'j']
    SUPRA_ANN = ['A', 'a', 'J', 'S']
    VENTRI_ANN = ['V', 'E']
    FUSION_ANN = ['F']
    UNCLASS_ANN = ['/', 'f', 'Q']
    
    
    """
    0 : N, 1 : S, 2 : V, 3 : F, 4 : Q, 5: "/"
    """
    for i,sym in enumerate(record_ann_sym):
        if sym in NORMAL_ANN:
            record_ann_sym[i] = 0
        elif sym in SUPRA_ANN:
            record_ann_sym[i] = 1
        elif sym in VENTRI_ANN:
            record_ann_sym[i] = 2
        elif sym in FUSION_ANN:
            record_ann_sym[i] = 3
        elif sym in UNCLASS_ANN:
            record_ann_sym[i] = 4
        else:
            record_ann_sym[i] = 5
    
    def beat_Padding(beat,w_size):
        beat_len = len(beat)
        if beat_len <= w_size:
            pd_size = (w_size - beat_len)//2
        pad_beat = np.pad(beat, (pd_size, pd_size), "constant", constant_values=0)
        
        if len(pad_beat) < w_size:
            pad_beat = np.pad(pad_beat, (0, w_size - len(pad_beat)), "constant", constant_values=0)
        elif len(pad_beat) > w_size:
            pad_beat = pad_beat[0:-(len(pad_beat)-w_size)]
        
        
        return pad_beat
    
    def beat_extraction(signal,rpeaks): # Peak를 기점으로 앞뒤로 0.3초 정도의 window를 가지는 Beat들
        beat = []
        for i,rp in enumerate(rpeaks):
            if i == 0: # 0.3초 정도의 window
                start = int(rp-(0.3*360))
                if start < 0:
                    start = 0
                end = int(rp+(0.3*360))
                sig_beat = beat_Padding(signal[start:end], window_size)
                beat.append(sig_beat)
                continue
            
            # 양쪽으로 30% 정도 남겨서 짜르기
            
            start = int(rp-(0.3 * 360))
            end = int(rp+(0.3 * 360))
            
            if end > len(signal):
                end = len(signal-1)
                
            # Beat Padding
            sig_beat = beat_Padding(signal[start:end], window_size)
            beat.append(sig_beat)

        return {"Beats":beat,"rpeaks":rpeaks, "symbol":record_ann}
    
    beat = beat_extraction(record_sig[0][:,0], record_ann)
    
    print("Beats, Rpeaks Len, ",len(beat["Beats"]),len(beat["rpeaks"]), len(record_ann))
    
    beat["symbol"]=record_ann_sym[:]
    max_len = -1
    min_len = 9999
    for i,datas in enumerate(zip(beat["rpeaks"],beat["symbol"])):
        max_len = max(max_len, len(beat["Beats"][i]))
        min_len = min(min_len, len(beat["Beats"][i]))
    return max_len, min_len, beat

datas = {}

for i,rec in enumerate(records_list):
    print("Record",rec)
    maxs, mins, beats = extract_from_sbj(i)
    datas[rec]=beats
    print(maxs,mins)

Record 100
Beats, Rpeaks Len,  2273 2273 2273
320 320
Record 101
Beats, Rpeaks Len,  1873 1873 1873
320 320
Record 102
Beats, Rpeaks Len,  2191 2191 2191
320 320
Record 103
Beats, Rpeaks Len,  2090 2090 2090
320 320
Record 104
Beats, Rpeaks Len,  2310 2310 2310
320 320
Record 105
Beats, Rpeaks Len,  2690 2690 2690
320 320
Record 106
Beats, Rpeaks Len,  2097 2097 2097
320 320
Record 107
Beats, Rpeaks Len,  2139 2139 2139
320 320
Record 108
Beats, Rpeaks Len,  1823 1823 1823
320 320
Record 109
Beats, Rpeaks Len,  2534 2534 2534
320 320
Record 111
Beats, Rpeaks Len,  2132 2132 2132
320 320
Record 112
Beats, Rpeaks Len,  2549 2549 2549
320 320
Record 113
Beats, Rpeaks Len,  1795 1795 1795
320 320
Record 114
Beats, Rpeaks Len,  1889 1889 1889
320 320
Record 115
Beats, Rpeaks Len,  1961 1961 1961
320 320
Record 116
Beats, Rpeaks Len,  2420 2420 2420
320 320
Record 117
Beats, Rpeaks Len,  1538 1538 1538
320 320
Record 118
Beats, Rpeaks Len,  2300 2300 2300
320 320
Record 119
Beats, Rpeaks Len

In [53]:
pickle_path = "./pickle"

In [54]:
# Save pickle File

import pickle

for rec in records_list:
    pickle_data = pickle_path + "/" + rec + ".pkl"
    with open(pickle_data,"wb") as f:
        pickle.dump(datas[rec], f, pickle.HIGHEST_PROTOCOL)

### Model




In [6]:
# Interpretation of Electrocardiogram Heartbeat by CNN and GRU (not Use)
class LFEM(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size):
        super(LFEM, self).__init__()
        
        self.lfem_seq = nn.Sequential(
            nn.Conv1d(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_size, stride=1, padding=1),
            nn.BatchNorm1d(out_channel),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
    
    def forward(self, x):
        return self.lfem_seq(x)

class LFEM_Stack(nn.Module):
    def __init__(self, num_layer=6):
        super(LFEM_Stack, self).__init__()
        
        self.num_layer = num_layer
        self.lfem_list = []
        layer_size=[64,64,128,128,256,256]
        for i in range(num_layer-1):
            self.lfem_list.append(LFEM(in_channel=layer_size[i], out_channel=layer_size[i+1], kernel_size=3))
        self.lfem_list.append(LFEM(in_channel=layer_size[5], out_channel=layer_size[5], kernel_size=3))
            
    def forward(self, x):
        for i in range(self.num_layer):
            x = self.lfem_list[i](x)
            
        return x
    

In [6]:
class conv_gru(nn.Module):
    def __init__(self, in_channel, out_channel, batch, hidden_size):
        super(conv_gru, self).__init__()
        
        #input Layer
        self.input_Seq1 = nn.Sequential(
            nn.Conv1d(in_channels=in_channel, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU()
        )
        self.input_Seq2 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        self.input_Seq3 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        self.input_Seq4 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )
#         self.lfem_module = LFEM_Stack(6)
        # reshape
        # dropout
        # gru
        # dense 1
        # dense 2
        self.dropout = nn.Dropout(0.3)
        self.gru = nn.GRU(input_size=320,batch_first=True, hidden_size=hidden_size ,num_layers=2, bidirectional=False)
        
        self.dense_Seq = nn.Sequential(
            nn.Linear(256*hidden_size, 256),
            nn.LeakyReLU(inplace=True),
            nn.Linear(256,out_channel)
        )
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        x = self.input_Seq1(x)
        x = self.input_Seq2(x)
        x = self.input_Seq3(x)
        x = self.input_Seq4(x)
        x = self.dropout(x)
        x, _ = self.gru(x)
        x = x.reshape(x.size(0),-1)
        print(x.shape)
        x = self.dense_Seq(x)
#         x = self.dense1(x)
#         x = self.dense2(x)
        
        return self.softmax(x)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = conv_gru(in_channel=1,out_channel=5,batch=32,hidden_size=256)
model.to(device)
# torch.shape : (batch, Channel, W(1채널의경우 1), H)
# print("Model output Shape, ",model(torch.randn(32,1,320)).shape) # test

conv_gru(
  (input_Seq1): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (input_Seq2): Sequential(
    (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (input_Seq3): Sequential(
    (0): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (input_Seq4): Sequential(
    (0): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (gru): GRU(320, 256, num_layers=2, batch_first=True)
  (dense_Seq): Sequential(
    (0): Linear(in_features=65536, ou

In [8]:
print(pytorch_model_summary.summary(model,torch.randn(32,1,320).cuda(), show_input=True, batch_size=32, show_hierarchical=True))

torch.Size([32, 65536])
-----------------------------------------------------------------------
      Layer (type)         Input Shape         Param #     Tr. Param #
          Conv1d-1        [32, 1, 320]             128             128
     BatchNorm1d-2       [32, 32, 320]              64              64
            ReLU-3       [32, 32, 320]               0               0
          Conv1d-4       [32, 32, 320]           6,208           6,208
     BatchNorm1d-5       [32, 64, 320]             128             128
            ReLU-6       [32, 64, 320]               0               0
          Conv1d-7       [32, 64, 320]          24,704          24,704
     BatchNorm1d-8      [32, 128, 320]             256             256
            ReLU-9      [32, 128, 320]               0               0
         Conv1d-10      [32, 128, 320]          98,560          98,560
    BatchNorm1d-11      [32, 256, 320]             512             512
           ReLU-12      [32, 256, 320]              

### Data Loader

In [7]:
# Pickle Data Load

def load_data():
    pkl_path = "./pickle/"

    data_x, data_y = [] ,[] 
    for pkl_file in os.listdir(pkl_path):
        with open(pkl_path+pkl_file, "rb") as f:
            data = pickle.load(f)
            for i,d in enumerate(data["Beats"]):
                data_x.append(d)
                data_y.append(data["symbol"][i])
#             print(torch.tensor(data["Beats"]).shape)
#             print(torch.tensor(data["symbol"]).shape)
    return torch.tensor(data_x), torch.tensor(data_y)

x_data, y_data = load_data()

In [8]:
x_data.shape

torch.Size([112599, 320])

In [9]:
y_data.shape

torch.Size([112599])

In [10]:
train_data = torch.utils.data.TensorDataset(x_data,y_data)
train_len = x_data.shape[0]
print(train_len)
val_len = int(train_len * 0.2)
print(val_len)
train_len -= val_len
print(train_len)
train_dataset, val_dataset = torch.utils.data.random_split(train_data, [train_len, val_len])

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

112599
22519
90080


In [11]:
lr = 3e-3
epochs = 50
torch.manual_seed(1234)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = conv_gru(in_channel=1,out_channel=5,batch=32,hidden_size=256)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

best_acc, best_epoch = 0, 0
global_step = 0

In [12]:
# def main():
train_dataset[0][0].shape

torch.Size([320])

In [13]:
# data 확인
# dataiter = iter(train_loader)
# sig, lab = dataiter.next()

In [14]:
# sig.shape

In [17]:
from tqdm import tqdm

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

for ep in range(epochs):
    train_bar = tqdm(train_loader)
    for step, (x,y) in enumerate(train_bar):
        
        x = x.unsqueeze(dim=1)
        print(x.shape, y.shape)
        x, y = x.to(device).float(), y.to(device)
        print(y)
        model.train()
        logits = model(x)
        print(logits.shape)
        loss = criterion(logits, y)
        print(loss)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step() 
        
        train_bar.desc = "Train Epoch[{}/{}] loss: {:.3f}".format(ep+1, epochs, loss)
        global_step +=1


  0%|                                                                                                                                                                     | 0/2815 [00:00<?, ?it/s]

torch.Size([32, 1, 320]) torch.Size([32])


RuntimeError: CUDA error: device-side assert triggered